In [1]:
from requests import get
import simplejson as json
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd

In [2]:
url = 'http://emojipedia.org'

In [3]:
def get_soup(url):
    r = get(url)
    return BeautifulSoup(r.text, 'html.parser')

In [4]:
def get_main_categories():
    soup = get_soup(url)
    catgeories_with_ref = defaultdict(str)
    for div in soup.findAll('div'):
        if 'block' in div.attrs.get('class', []) and div.h2 and div.h2.text == 'Categories':
            for page in div.findAll('li'):
                category_url_ref = page.find('a').get('href')
                category = page.find('a').text
                if not 'http://' in category_url_ref:
                    catgeories_with_ref[category_url_ref] = category
    return catgeories_with_ref

In [5]:
def get_category_with_urls(categories_with_ref):
    cat_with_urls = defaultdict(list)
    for url_ext, cat in list(categories_with_ref.items()):
        mini_soup = get_soup(url + url_ext)
        divs = mini_soup.findAll('div')
        clean_cat = ' '.join(cat.split()[1:])
        for div in divs:
            if div.h1:
                name = ' '.join(div.h1.text.strip().split()[1:])
                if name in clean_cat:
                    for l in div.findAll('li'):
                        if not 'http://' in l.find('a').get('href'):
                            cat_with_urls[cat] += [l.find('a').get('href')]
    return cat_with_urls

In [6]:
def extract_emoji_info(url):
    soup = get_soup(url)
    # TITLE
    title = soup.h1.text
    # DESCRIPTION
    sections = soup.findAll('section')
    aliases = []
    description = ''
    for section in sections:
        name = section.get('class')[0]
        if name == 'description':
            description = section.p.text
        if name == 'aliases':
            
            for alias_item in sections[1].findAll('li'):
                aliases.append(' '.join(alias_item.text.split()[1:]))

    section = soup.find("section",{"class" : "vendor-list"})
    
    for vendor in section.findAll('li'):
        
        

    
                
    shortcodes = []
    related = []
    for l in soup.findAll('ul'):
        # ShortCodes
        cl = l.get('class', [''])[0]
        if cl == 'shortcodes':
            shortcodes = [shortcode_nest.text for shortcode_nest in l.findAll('li')]
        # RELATED
        if cl == 'emoji-list':
            related = [related_nest.get('href') for related_nest in l.findAll('li')]
        # CODE POINTS
            
    return {'name': title if title else ' '.join(title.split()[1:]),
            'clean_name': ' '.join(title.split()[1:]),
            'description': description,
            'alternate_names': aliases,
            'shortcodes': shortcodes,
            'url': url
            }

In [7]:
def extract_all_page_info(cat_with_urls):
    data = []
    for cat, urls in list(cat_with_urls.items()):
        for url_ext in list(set(urls)):
            extracted = extract_emoji_info(url + url_ext)
            if extracted.get('name') != '':
                data.append(extracted)
    data = pd.DataFrame(data)
    data.to_csv('data/emoji_lookup.csv')
    return data

In [8]:
def main():
    categories_with_ref = get_main_categories()
    cat_with_urls = get_category_with_urls(categories_with_ref)
    data = extract_all_page_info(cat_with_urls)

In [9]:
main()

In [39]:
url = 'http://emojipedia.org/roller-coaster/'
soup = get_soup(url)
section = soup.find("section",{"class" : "vendor-list"})

In [22]:
list_ = section.findAll('li')

In [33]:
print(list_[0].find("div", {"class" : "vendor-info"}, "href").text)
print(list_[0].find("img")["src"])


Apple

http://emojipedia-us.s3.amazonaws.com/cache/2c/2e/2c2e95b48ff1c164273b1815a3c6d6c1.png


In [36]:
list_[2]

<li>
<div class="vendor-container">
<div class="vendor-info">
<p class="version-name"><a href="/apple/ios-9.1/">iOS 9.1</a></p>
</div>
<div class="vendor-image">
<a href="/apple/ios-9.1/roller-coaster/">
<img alt="Roller Coaster on Apple " src="http://emojipedia-us.s3.amazonaws.com/cache/70/f4/70f4ccd4ce9f07e50c9bc9380489c848.png">
</img></a>
</div>
</div>
</li>

In [18]:
list_[0]

<li>
<div class="vendor-container vendor-rollout-target">
<div class="vendor-info">
<h2><a href="/apple/">Apple</a></h2>
</div>
<div class="vendor-image">
<img alt="Roller Coaster on Apple iOS 9.3" src="http://emojipedia-us.s3.amazonaws.com/cache/2c/2e/2c2e95b48ff1c164273b1815a3c6d6c1.png">
</img></div>
</div>
<div class="vendor-rollout-container">
<ul class="vendor-rollout">
<li>
<div class="vendor-container">
<div class="vendor-info">
<p class="version-name"><a href="/apple/ios-9.3/">iOS 9.3</a></p>
</div>
<div class="vendor-image">
<a href="/apple/ios-9.3/roller-coaster/">
<img alt="Roller Coaster on Apple " src="http://emojipedia-us.s3.amazonaws.com/cache/32/4a/324a614f4fd5e0659fb64642b685d495.png">
</img></a>
</div>
</div>
</li>
<li>
<div class="vendor-container">
<div class="vendor-info">
<p class="version-name"><a href="/apple/ios-9.1/">iOS 9.1</a></p>
</div>
<div class="vendor-image">
<a href="/apple/ios-9.1/roller-coaster/">
<img alt="Roller Coaster on Apple " src="http://emoj